In [57]:
!pip install redis


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for redis from https://files.pythonhosted.org/packages/0b/34/a01250ac1fc9bf9161e07956d2d580413106ce02d5591470130a25c599e3/redis-5.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/250.3 kB ? eta -:--:--
   ------ -------------------------------- 41.0/250.3 kB 991.0 kB/s eta 0:00:01
   ------------------ --------------------- 112.6/250.3 kB 1.6 MB/s eta 0:00:01
   --------------------------- ------------ 174.1/250.3 kB 1.3 MB/s eta 0:00:01
   ------------------------------------ --- 225.3/250.3 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 250.3/250.3 kB 1.1 MB/s eta 0:00:00


In [6]:
import redis
import os
from deepface import DeepFace
import numpy as np

Redis Database on Redis Cloud


In [7]:
r = redis.Redis(
    host="redis-16765.c1.us-central1-2.gce.cloud.redislabs.com",
    port=16765,
    password="0iZGj6DxsJq8bjZ2NhctmWBpCPvX0Hu7",
    ssl=False,
)

Redis Database on My Local (Ubuntu 22.04)


In [4]:
r = redis.Redis(
    host="10.101.0.12",
    port=6380,
    # password="0iZGj6DxsJq8bjZ2NhctmWBpCPvX0Hu7",
    ssl=False,
)

In [8]:
r.ping()

True

In [9]:
embeddings = []
for dirpath, dirnames, filenames in os.walk("C:/Users/lenovo/Downloads/test_dataset/"):
    for filename in filenames:
        img_path = f"{dirpath}{filename}"
        if ".jpg" in img_path:
            # print(img_path)
            embedding = DeepFace.represent(
                img_path=img_path, detector_backend="yolov8"
            )[0]["embedding"]
            embeddings.append((img_path, embedding))

In [10]:
embeddings[1][0].split("/")

['C:', 'Users', 'lenovo', 'Downloads', 'test_dataset', 'moad.jpg']

In [11]:
len(embeddings)

8

In [12]:
pipeline = r.pipeline(transaction=False)
for img_path, embedding in embeddings:
    key = img_path.split("/")[-1]
    value = np.array(embedding).astype(np.float32).tobytes()

    pipeline.hset(key, mapping={"embedding": value, "NAME": "Omar"})

pipeline_result = pipeline.execute()

In [6]:
from redis.commands.search.field import VectorField

In [485]:
len(embeddings[0][1])

2622

In [7]:
r.ft().info("INDEX", "idx")

TypeError: SearchCommands.info() takes 1 positional argument but 3 were given

In [8]:
# Clear existing index
r.execute_command("FT.DROPINDEX", "idx")

b'OK'

In [492]:
r.ft().info()

{'index_name': 'idx',
 'index_options': [],
 'index_definition': [b'key_type',
  b'HASH',
  b'prefixes',
  [b''],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'embedding',
   b'attribute',
   b'embedding',
   b'type',
   b'VECTOR',
   b'algorithm',
   b'HNSW',
   b'data_type',
   b'FLOAT32',
   b'dim',
   2622,
   b'distance_metric',
   b'L2',
   b'M',
   16,
   b'ef_construction',
   200]],
 'num_docs': '8',
 'max_doc_id': '8',
 'num_terms': '0',
 'num_records': '8',
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '2.9943456649780273',
 'total_inverted_index_blocks': '0',
 'offset_vectors_sz_mb': '0',
 'doc_table_size_mb': '7.524490356445313e-4',
 'sortable_values_size_mb': '0',
 'key_table_size_mb': '3.0422210693359375e-4',
 'geoshapes_sz_mb': '0',
 'records_per_doc_avg': '1',
 'bytes_per_record_avg': '0',
 'offsets_per_term_avg': '0',
 'offset_bits_per_record_avg': 'nan',
 'hash_indexing_failures': '0',
 'total_indexing_time': '0.35899999737739563',
 'indexing': '

In [490]:
r.ft().create_index(
    [
        VectorField(
            "embedding",
            "HNSW",
            {
                "TYPE": "FLOAT32",
                "DIM": 2622,
                "DISTANCE_METRIC": "L2",
            },
        )
    ]
)

b'OK'

In [491]:
r.execute_command("FT.LIST")

ResponseError: unknown command 'FT.LIST'

In [424]:
target_img = r"C:\Users\lenovo\Downloads\dbs\omar.jpg"

In [425]:
target_embedding = DeepFace.represent(img_path=target_img, detector_backend="yolov8")[
    0
]["embedding"]

In [426]:
query_vector = np.array(target_embedding).astype(np.float32).tobytes()

In [420]:
from redis.commands.search.query import Query

In [430]:
k = 8
base_query = f"*=>[KNN {k} @embedding $query_vector AS distance]"
query = Query(base_query).return_fields("distance").sort_by("distance").dialect(2)
results = r.ft().search(query, query_params={"query_vector": query_vector})

In [460]:
for result in results.docs:
    print(result)

Document {'id': 'WhatsApp Image 2024-01-30 at 20.15.32_09726ec2.jpg', 'payload': None, 'distance': '0.111752577126'}
Document {'id': 'WhatsApp Image 2024-01-29 at 22.12.45_43ccb782.jpg', 'payload': None, 'distance': '0.12307536602'}
Document {'id': 'WhatsApp Image 2024-01-29 at 22.12.45_219fe705.jpg', 'payload': None, 'distance': '0.139388933778'}
Document {'id': 'moad_pass.jpg', 'payload': None, 'distance': '0.163798689842'}
Document {'id': '20240123_203124.jpg', 'payload': None, 'distance': '0.165776208043'}
Document {'id': 'WhatsApp Image 2024-01-10 at 11.04.14_0380b28b.jpg', 'payload': None, 'distance': '0.241512924433'}
Document {'id': 'moad.jpg', 'payload': None, 'distance': '0.268328905106'}
Document {'id': 'omar_moad.jpg', 'payload': None, 'distance': '0.434868633747'}


In [464]:
for i in results.docs:
    print(i.id, "=>", round(100 - (float(i.distance) * 100), 2), "%")

WhatsApp Image 2024-01-30 at 20.15.32_09726ec2.jpg => 88.82 %
WhatsApp Image 2024-01-29 at 22.12.45_43ccb782.jpg => 87.69 %
WhatsApp Image 2024-01-29 at 22.12.45_219fe705.jpg => 86.06 %
moad_pass.jpg => 83.62 %
20240123_203124.jpg => 83.42 %
WhatsApp Image 2024-01-10 at 11.04.14_0380b28b.jpg => 75.85 %
moad.jpg => 73.17 %
omar_moad.jpg => 56.51 %
